In [ ]:
# установка дополнительных библиотек
!pip install easydict neptune-client
!pip install jsonschema==4.4.0
!pip install faiss-gpu
!pip install setuptools==40.6.2

In [ ]:
!git clone -b XBM https://github.com/pkhanzhina/ImagerRetrieval.git

In [ ]:
%cd "ImagerRetrieval/research-xbm"

In [ ]:
!python setup.py develop build

In [ ]:
# подгрузка преобученной модели ResNet-50
!wget -O resnet50-0676ba61.pth https://download.pytorch.org/models/resnet50-0676ba61.pth

In [ ]:
# конфиг обучения
import yaml

config = {
    'NAME': 'triplet_loss_0_4',
    'MODEL': {
        'BACKBONE': {'NAME': 'resnet50'},
        'HEAD': {'DIM': 128, 'IN_CHANNELS': 2048}
    },
    'LOSSES': {'NAME': 'triplet_loss'},
    'INPUT': {
        'MODE': 'RGB',
        'PIXEL_MEAN': [0.485, 0.456, 0.406],
        'PIXEL_STD': [0.229, 0.224, 0.225],
    },
    'SOLVER': {
        'MAX_ITERS': 10000,
        'STEPS': 2,
        'OPTIMIZER_NAME': 'Adam',
        'BASE_LR': 1e-5,
        'WEIGHT_DECAY': 0.0005,
        'CHECKPOINT_PERIOD': 182
    },
    'DATA': {
        'TRAIN_IMG_SOURCE': '../../../input/flowersdataset/annotation_train.csv',
        'TEST_IMG_SOURCE': '../../../input/flowersdataset/annotation_test.csv',
        'TRAIN_BATCHSIZE': 64,
        'TEST_BATCHSIZE': 64,
        'NUM_WORKERS': 4,
        'NUM_INSTANCES': 4,
    },
    'VALIDATION': {'VERBOSE': 182},
    'XBM': {
        'ENABLE': True,
        'WEIGHT': 0.5,
        'SIZE': 15000,
        'START_ITERATION': 182,
    },
    'run_id': None,
}

with open('sample_config.yaml', 'w') as outfile:
    yaml.dump(config, outfile, default_flow_style=False)

In [ ]:
# запуск обучения
!python tools/train_net.py --cfg sample_config.yaml